In [ ]:
import os
import cv2
import numpy as np
import imageio
import yt_dlp
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip
import warnings
warnings.filterwarnings("ignore")

# Function to download video using yt-dlp
def download_video(url, output_filename):
    ydl_opts = {
        'outtmpl': output_filename,
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]'
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

# Function to extract keyframes
def extract_keyframes(video_path, confidence=0.5, target_size=(480, 360)):
    if not os.path.exists(video_path):
        print(f"Error: Cannot open {video_path}")
        return []

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open {video_path}")
        return []

    frames = []
    prev_frame = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_resized = cv2.resize(frame, target_size)

        if prev_frame is None:
            frames.append(frame_resized)
        else:
            diff = cv2.absdiff(cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY),
                               cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY))
            score = np.mean(diff)

            if score > (confidence * 255):
                frames.append(frame_resized)

        prev_frame = frame_resized

    cap.release()
    print(f"Extracted {len(frames)} keyframes from {video_path}")
    return frames

# Function to display extracted frames
def display_frames(frames, title="Extracted Keyframes"):
    if not frames:
        print("No frames to display.")
        return

    fig, axes = plt.subplots(1, min(5, len(frames)), figsize=(15, 5))
    if len(frames) == 1:
        axes = [axes]  # Ensure axes is iterable when only one frame is present

    for ax, frame in zip(axes, frames[:5]):  # Display up to 5 keyframes
        ax.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        ax.axis("off")

    plt.suptitle(title)
    plt.show()

# Function to save summary video
def save_summary_video(frames, output_filename, fps=10):
    if not frames:
        print("No frames to save.")
        return

    writer = imageio.get_writer(output_filename, fps=fps)

    for frame in frames:
        writer.append_data(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    writer.close()
    print(f"Summary video saved: {output_filename}")

# Download two videos
download_video("https://www.youtube.com/watch?v=ocPxX7NrcZQ", "video1.mp4")
download_video("https://www.youtube.com/watch?v=XZ5bGEhTKMg", "video2.mp4")

# Extract keyframes from both videos
frames_video1 = extract_keyframes("video1.mp4", confidence=0.5)
frames_video2 = extract_keyframes("video2.mp4", confidence=0.5)

# Display extracted keyframes
display_frames(frames_video1, title="Keyframes from Video 1")
display_frames(frames_video2, title="Keyframes from Video 2")

# Merge keyframes from both videos
summary_frames = frames_video1 + frames_video2

# Display summary frames
display_frames(summary_frames, title="Summary Keyframes")

# Save summary video
save_summary_video(summary_frames, "summary.mp4")


In [ ]:
import torch
import cv2
import numpy as np
import joblib
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

# Load YOLO model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Function to apply YOLO and extract features
def detect_objects(frames):
    feature_data = []

    for i, frame in enumerate(frames):
        results = model(frame)
        detected_objects = results.pandas().xyxy[0]['name'].tolist()

        # Convert object list to numerical features
        object_counts = {obj: detected_objects.count(obj) for obj in set(detected_objects)}
        feature_vector = [
            object_counts.get('person', 0),
            object_counts.get('knife', 0),
            object_counts.get('gun', 0),
            object_counts.get('cell phone', 0),
            object_counts.get('backpack', 0)
        ]
        feature_data.append(feature_vector)

    return feature_data

# Extract features from keyframes
features = detect_objects(summary_frames)

# Load or train a decision tree classifier
try:
    classifier = joblib.load("crime_classifier.pkl")
except FileNotFoundError:
    X_train = np.random.randint(0, 5, (50, 5))  # Simulated training data
    y_train = np.random.choice([0, 1], size=50)  # 0: No crime, 1: Crime detected
    classifier = DecisionTreeClassifier()
    classifier.fit(X_train, y_train)
    joblib.dump(classifier, "crime_classifier.pkl")

# Predict crime occurrence
predictions = classifier.predict(features)
crime_detected = np.mean(predictions) > 0.5  # Decision based on majority

print("Crime detected: shooting" if crime_detected else "No crime detected.")

